In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Load Data

In [2]:
trainFile2018 = 'C:\\Users\\hycwy\\Desktop\\Python data\\Stock 200 indicator\\2018_Financial_Data.csv'
trainFile2017 = 'C:\\Users\\hycwy\\Desktop\\Python data\\Stock 200 indicator\\2017_Financial_Data.csv'

pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile2018))
trainData1 = pd.read_csv(os.path.basename(trainFile2018))
trainData2 = pd.read_csv(os.path.basename(trainFile2017))
os.chdir(pwd)

trainData = pd.concat([trainData1,trainData2])


# Preprocessing

In [3]:

data = trainData1.iloc[0:4392,0:225] #when use larger dataset[0:9353,0:224], model performs worse
data = data.fillna(data.mean())

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder().fit_transform(data['Sector'])
ohe = OneHotEncoder(sparse=False).fit_transform(le.reshape(-1,1))
enc = ohe.astype(int)
sector_new = pd.DataFrame(enc) 

data_class = data['Class']
data_drop = data.drop(['Sector','Class'],axis=1)

data = pd.concat([data_drop,sector_new],axis=1)



In [4]:
X = data.iloc[:,1:235]
y = data_class

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)


from sklearn.preprocessing import StandardScaler
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

In [5]:
from sklearn.svm import LinearSVC
lclf = LinearSVC().fit(X_train_scaled,y_train)
print('accuracy on training set: %f' % lclf.score(X_train_scaled,y_train))
print('accuracy on test set: %f' % lclf.score(X_test_scaled,y_test))

accuracy on training set: 0.996357
accuracy on test set: 0.925319


C:\Users\hycwy\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# SVC 

In [6]:
from sklearn.svm import SVC
clf = SVC(C=100, kernel='rbf', gamma=0.01).fit(X_train_scaled,y_train)

print('accuracy on training set: %f' % clf.score(X_train_scaled,y_train))
print('accuracy on test set: %f' % clf.score(X_test_scaled,y_test))

accuracy on training set: 0.999696
accuracy on test set: 0.903461


In [7]:
def svc_func(C,gamma):
    svcclf = SVC(C=C,gamma=gamma,kernel='rbf').fit(X_train_scaled,y_train)
    return svcclf.score(X_test_scaled,y_test)



In [8]:
from bayes_opt import BayesianOptimization

pbounds = {'C':(1,100),'gamma':(0.01,0.1)}
optimizersvc = BayesianOptimization(
    f=svc_func,
    pbounds=pbounds,
    verbose=2,
    random_state=0,
)
optimizersvc.maximize(n_iter=10,init_points=5)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.8124   |  55.33    |  0.07437  |
|  2        |  0.8306   |  60.67    |  0.05904  |
|  3        |  0.8133   |  42.94    |  0.06813  |
|  4        |  0.7878   |  44.32    |  0.09026  |
|  5        |  0.8552   |  96.4     |  0.04451  |
|  6        |  0.7823   |  95.33    |  0.1      |
|  7        |  0.8634   |  96.4     |  0.04079  |
|  8        |  0.8151   |  83.32    |  0.06759  |
|  9        |  0.8479   |  96.37    |  0.04798  |
|  10       |  0.9035   |  96.42    |  0.01403  |
|  11       |  0.9007   |  96.44    |  0.01202  |
|  12       |  0.8552   |  96.41    |  0.04475  |
|  13       |  0.7942   |  9.078    |  0.08329  |
|  14       |  0.8843   |  96.52    |  0.02277  |
|  15       |  0.8862   |  83.23    |  0.02514  |


# NN

In [9]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='adam',random_state=0,hidden_layer_sizes=[100,100],max_iter=300,activation='logistic',learning_rate_init=0.1,learning_rate='adaptive',alpha=0.01).fit(X_train,y_train)

print('accuracy on training set: %f' % mlp.score(X_train_scaled,y_train))
print('accuracy on test set: %f' % mlp.score(X_test_scaled,y_test))

accuracy on training set: 0.470249
accuracy on test set: 0.476321


In [10]:
from bayes_opt import BayesianOptimization

def ml_func1(
            hidden_layer_sizes1,
            max_iter,
            learning_rate_init,
            alpha):
    
    layer1 = int(hidden_layer_sizes1)
    
    ml = MLPClassifier(solver='adam',
                       random_state=0,
                       hidden_layer_sizes=[layer1],
                       max_iter=int(max_iter),
                       activation='logistic',
                       learning_rate_init=learning_rate_init,
                       learning_rate='adaptive',
                       alpha=alpha).fit(X_train,y_train)
    return ml.score(X_train,y_train)



pbounds = {'max_iter':(100,600),'learning_rate_init':(0.01,1),'alpha':(0.01,1),'hidden_layer_sizes1':(100,300)}
optimizer1 = BayesianOptimization(
    f=ml_func1,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [11]:
optimizer1.maximize(n_iter=50,init_points=5)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
|  1        |  0.7092   |  0.4229   |  244.1    |  0.01011  |  251.2    |
|  2        |  0.3254   |  0.1553   |  118.5    |  0.1944   |  272.8    |
|  3        |  0.6627   |  0.4028   |  207.8    |  0.425    |  442.6    |
|  4        |  0.7043   |  0.2124   |  275.6    |  0.03711  |  435.2    |
|  5        |  0.6885   |  0.4231   |  211.7    |  0.149    |  199.1    |
|  6        |  0.6946   |  0.9182   |  242.3    |  0.5583   |  252.3    |
|  7        |  0.7086   |  0.01     |  271.5    |  0.01     |  217.0    |
|  8        |  0.7137   |  0.01     |  290.3    |  0.01     |  265.1    |
|  9        |  0.5932   |  0.05791  |  295.6    |  0.1394   |  371.8    |
|  10       |  0.6916   |  0.1975   |  272.2    |  0.4901   |  492.3    |
|  11       |  0.6463   |  0.02716  |  243.2    |  0.7264   |  141.5    |
|  12       |  0.7028   |  1.0      | 

In [12]:

def ml_func2(
            hidden_layer_sizes1,
            hidden_layer_sizes2,
            max_iter,
            learning_rate_init,
            alpha):
    
    layer1 = int(hidden_layer_sizes1)
    layer2 = int(hidden_layer_sizes2)
    ml = MLPClassifier(solver='adam',
                       random_state=0,
                       hidden_layer_sizes=[layer1,layer2],
                       max_iter=int(max_iter),
                       activation='logistic',
                       learning_rate_init=learning_rate_init,
                       learning_rate='adaptive',
                       alpha=alpha).fit(X_train,y_train)
    return ml.score(X_train,y_train)



pbounds = {'max_iter':(100,600),'learning_rate_init':(0.01,1),'alpha':(0.01,1),'hidden_layer_sizes1':(100,300),'hidden_layer_sizes2':(200,400)}
optimizer2 = BayesianOptimization(
    f=ml_func2,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [14]:
optimizer2.maximize(n_iter=20,init_points=5)

|   iter    |  target   |   alpha   | hidden... | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------------------
|  6        |  0.6916   |  0.8957   |  117.0    |  207.8    |  0.1781   |  539.1    |
|  7        |  0.3084   |  0.1074   |  184.2    |  391.6    |  0.5378   |  445.9    |
|  8        |  0.7031   |  0.3224   |  237.3    |  366.9    |  0.02811  |  475.1    |
|  9        |  0.6916   |  0.989    |  249.6    |  256.1    |  0.7914   |  151.6    |
|  10       |  0.6916   |  0.4534   |  281.7    |  258.7    |  0.2949   |  165.0    |
|  11       |  0.6916   |  0.4889   |  251.5    |  254.9    |  0.5075   |  151.7    |
|  12       |  0.6627   |  0.5169   |  285.9    |  345.3    |  0.01     |  501.1    |
|  13       |  0.7046   |  0.01     |  229.6    |  252.0    |  0.01     |  239.2    |
|  14       |  0.7064   |  0.01     |  166.9    |  229.1    |  0.01     |  198.7    |
|  15       |  0.6916   |  0.9384   |  161.2    |  256

In [15]:
def ml_func3(
            hidden_layer_sizes1,
            hidden_layer_sizes2,
            hidden_layer_sizes3,
            max_iter,
            learning_rate_init,
            alpha):
    
    layer1 = int(hidden_layer_sizes1)
    layer2 = int(hidden_layer_sizes2)
    layer3 = int(hidden_layer_sizes3)
    ml = MLPClassifier(solver='adam',
                       random_state=0,
                       hidden_layer_sizes=[layer1,layer2,layer3],
                       max_iter=int(max_iter),
                       activation='logistic',
                       learning_rate_init=learning_rate_init,
                       learning_rate='adaptive',
                       alpha=alpha).fit(X_train,y_train)
    return ml.score(X_train,y_train)



pbounds = {'max_iter':(100,600),'learning_rate_init':(0.01,1),'alpha':(0.01,1),'hidden_layer_sizes1':(100,300),'hidden_layer_sizes2':(200,400),'hidden_layer_sizes3':(100,300)}
optimizer3 = BayesianOptimization(
    f=ml_func3,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

In [16]:
optimizer3.maximize(n_iter=20,init_points=10)

|   iter    |  target   |   alpha   | hidden... | hidden... | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------------------------------
|  1        |  0.6916   |  0.4229   |  244.1    |  200.0    |  160.5    |  0.1553   |  146.2    |
|  2        |  0.6916   |  0.1944   |  169.1    |  279.4    |  207.8    |  0.425    |  442.6    |
|  3        |  0.6916   |  0.2124   |  275.6    |  205.5    |  234.1    |  0.4231   |  379.3    |
|  4        |  0.6916   |  0.149    |  139.6    |  360.1    |  293.7    |  0.3203   |  446.2    |
|  5        |  0.6916   |  0.8776   |  278.9    |  217.0    |  107.8    |  0.1781   |  539.1    |
|  6        |  0.3084   |  0.1074   |  184.2    |  391.6    |  206.6    |  0.695    |  257.8    |
|  7        |  0.3084   |  0.6896   |  266.9    |  203.7    |  250.0    |  0.989    |  474.1    |
|  8        |  0.6916   |  0.2876   |  257.9    |  220.6    |  189.6    |  0.9095   |  246.8    |
|  9        |  0.691

# Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=10,random_state=3,bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None, max_features='auto', max_leaf_nodes=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_jobs=1,
 oob_score=False, verbose=0, warm_start=False).fit(X_train,y_train)

print('accuracy on training set: %f' %forest.score(X_train,y_train))
print('accuracy on test set: %f' %forest.score(X_test,y_test))

accuracy on training set: 1.000000
accuracy on test set: 0.988160


# Features Extraction

In [18]:
index_ = X_train.columns
weight_ = forest.feature_importances_
len(index_)==len(weight_)

True

In [19]:
d={}
for i in (range(233)):
    d[index_[i]]=weight_[i]
    


In [20]:

n = 10
 
L = sorted(d.items(),key=lambda item:item[1],reverse=True)
 
L = L[:n]
 
print(L)
 
dictdata = {}
for l in L:
    dictdata[l[0]] = l[1]
 
print (dictdata)

[('2019 PRICE VAR [%]', 0.48790037143622145), ('Consolidated Income', 0.017600775788965267), ('Total non-current liabilities', 0.012396380049736214), ('Invested Capital', 0.012185698865949076), ('Total assets', 0.01142655644374993), ('Total current liabilities', 0.011240856284284154), ('Total non-current assets', 0.010867583168413039), ('priceToOperatingCashFlowsRatio', 0.007983642705560134), (4, 0.005309117242912908), ('POCF ratio', 0.005277244724984531)]
{'2019 PRICE VAR [%]': 0.48790037143622145, 'Consolidated Income': 0.017600775788965267, 'Total non-current liabilities': 0.012396380049736214, 'Invested Capital': 0.012185698865949076, 'Total assets': 0.01142655644374993, 'Total current liabilities': 0.011240856284284154, 'Total non-current assets': 0.010867583168413039, 'priceToOperatingCashFlowsRatio': 0.007983642705560134, 4: 0.005309117242912908, 'POCF ratio': 0.005277244724984531}


In [21]:
new_indicators = []

In [22]:
for i in dictdata.keys():
    new_indicators.append(i)

new_indicators

['2019 PRICE VAR [%]',
 'Consolidated Income',
 'Total non-current liabilities',
 'Invested Capital',
 'Total assets',
 'Total current liabilities',
 'Total non-current assets',
 'priceToOperatingCashFlowsRatio',
 4,
 'POCF ratio']

In [23]:
X_new = X[new_indicators]

In [24]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new,y,random_state=11)

In [25]:
X_train_scaled_new = StandardScaler().fit_transform(X_train_new)
X_test_scaled_new = StandardScaler().fit_transform(X_test_new)

# SVM with Bayesian_opt

In [26]:
clf_new = SVC(C=100, kernel='rbf', gamma=0.001).fit(X_train_scaled_new,y_train_new)

print('accuracy on training set: %f' % clf_new.score(X_train_scaled_new,y_train_new))
print('accuracy on test set: %f' % clf_new.score(X_test_scaled_new,y_test_new))

accuracy on training set: 0.984821
accuracy on test set: 0.976321


In [27]:
def SVC_func_new(C,gamma):
    clf_new1 = SVC(C=C,kernel='rbf',gamma=gamma).fit(X_train_scaled_new,y_train_new)
    return clf_new1.score(X_test_scaled_new,y_test_new)


In [28]:
pbounds_new = {'C':(0.001,1000),'gamma':(0.001,1000)}
optimizersvc = BayesianOptimization(
    f=SVC_func_new,
    pbounds=pbounds_new,
    verbose=2,
    random_state=0,
)
optimizersvc.maximize(n_iter=50,init_points=10)
# weight of RF; features: 10,30,50,70; 

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.6867   |  548.8    |  715.2    |
|  2        |  0.6876   |  602.8    |  544.9    |
|  3        |  0.6876   |  423.7    |  645.9    |
|  4        |  0.6867   |  437.6    |  891.8    |
|  5        |  0.7386   |  963.7    |  383.4    |
|  6        |  0.6903   |  791.7    |  528.9    |
|  7        |  0.6867   |  568.0    |  925.6    |
|  8        |  0.8689   |  71.04    |  87.13    |
|  9        |  0.6867   |  20.22    |  832.6    |
|  10       |  0.6867   |  778.2    |  870.0    |
|  11       |  0.9718   |  271.5    |  0.001    |
|  12       |  0.8689   |  74.27    |  85.87    |
|  13       |  0.9663   |  518.4    |  0.001    |
|  14       |  0.9117   |  383.3    |  7.016    |
|  15       |  0.9654   |  728.9    |  0.001    |
|  16       |  0.8525   |  623.7    |  136.4    |
|  17       |  0.9663   |  1e+03    |  0.001    |
|  18       |  0.6867   |  0.001    |  375.3    |


In [35]:
def SVC_func_new_sig(C,gamma):
    clf_new1_sig = SVC(C=C,kernel='sigmoid',gamma=gamma).fit(X_train_scaled_new,y_train_new)
    return clf_new1_sig.score(X_test_scaled_new,y_test_new)

In [36]:
pbounds_new_sig = {'C':(10,150),'gamma':(0.001,1)}
optimizersvc_sig = BayesianOptimization(
    f=SVC_func_new_sig,
    pbounds=pbounds_new_sig,
    verbose=2,
    random_state=0,
)
optimizersvc_sig.maximize(n_iter=25,init_points=5) 

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.5865   |  86.83    |  0.7155   |
|  2        |  0.5874   |  94.39    |  0.5453   |
|  3        |  0.5883   |  69.31    |  0.6462   |
|  4        |  0.5656   |  71.26    |  0.8919   |
|  5        |  0.582    |  144.9    |  0.3841   |
|  6        |  0.5911   |  11.28    |  0.258    |
|  7        |  0.5537   |  86.82    |  0.7467   |
|  8        |  0.5893   |  126.4    |  0.6402   |
|  9        |  0.5856   |  93.94    |  0.2727   |
|  10       |  0.561    |  86.21    |  0.8231   |
|  11       |  0.5865   |  103.6    |  0.5311   |
|  12       |  0.5628   |  34.88    |  0.9207   |
|  13       |  0.5601   |  21.42    |  0.8145   |
|  14       |  0.5628   |  82.26    |  0.9502   |
|  15       |  0.602    |  126.3    |  0.169    |
|  16       |  0.5628   |  61.34    |  0.9874   |
|  17       |  0.5865   |  23.01    |  0.5579   |
|  18       |  0.5638   |  143.3    |  0.8435   |


In [29]:
def SVC_func_new_lin(C,gamma):
    clf_new1_lin = SVC(C=int(C),kernel='linear',gamma=gamma).fit(X_train_scaled_new,y_train)
    return clf_new1_lin.score(X_test_scaled_new,y_test)

In [ ]:
pbounds_new_lin = {'C':(10,150),'gamma':(0.001,1)}
optimizersvc_lin = BayesianOptimization(
    f=SVC_func_new_lin,
    pbounds=pbounds_new_lin,
    verbose=2,
    random_state=0,
)
optimizersvc_lin.maximize(n_iter=10,init_points=3) 

# RandomForest 

In [33]:
forest_new = RandomForestClassifier(n_estimators=100,random_state=3,bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None, max_features='auto', max_leaf_nodes=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_jobs=1,
 oob_score=False, verbose=0, warm_start=False).fit(X_train_new,y_train_new)

print('accuracy on training set: %f' %forest_new.score(X_train_new,y_train_new))
print('accuracy on test set: %f' %forest_new.score(X_test_new,y_test_new))

accuracy on training set: 1.000000
accuracy on test set: 0.999089
